In [1]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

d:\CodeField\Codefile\srp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [27]:
from transformers import pipeline

In [21]:
## the classifier is used to NER.
## 当然这东西花了十多分钟随便搭起来的，简陋了一点
class AbstractClassifier(object):
        
        def __init__(self,model_name,tokenizer_name,token_type):
            self.classifier = pipeline("token-classification",model=model_name,tokenizer=tokenizer_name)
            self.token_type = token_type
        # it will return a index_buf
        # item in index_buf explain
        # index_buf[0]: word start index
        # index_buf[1]: word end index
        # index_buf[2]: word types
        def get_sentence_index(self,sentence):
            ## 感兴趣的可以自行看看result的返回值，或许对你有所帮助
            dic_result = self.classifier(sentence)

            index_buf = []
            count = -1
            for item in dic_result:
                if( 'b-' in item['entity'].lower()):
                    count += 1
                    index_buf.append([item['start'],item['end'],self.token_type])
                if( 'i-' in item['entity'].lower()):
                    index_buf[count][1] = item['end']
            return index_buf


In [ ]:
## 用来识别 疾病实体 的分类器
disease_classifier = AbstractClassifier(model_name="alvaroalon2/biobert_diseases_ner",
                                        tokenizer_name="alvaroalon2/biobert_diseases_ner",
                                        token_type="disease")
## 用来识别 药物/化学品实体 的分类器
chemical_classifier = AbstractClassifier(model_name="sschet/biobert_chemical_ner",
                                         tokenizer_name="sschet/biobert_chemical_ner",
                                         token_type="chemical")
## 用来识别 基因/蛋白质实体 的分类器
genetic_classifier = AbstractClassifier(model_name="alvaroalon2/biobert_genetic_ner",
                                        tokenizer_name="alvaroalon2/biobert_genetic_ner",
                                        token_type="genetic")
## 测试句子
sentence = "Hepatitis B virus (Hepatitis B) is the pathogen that causes hepatitis B (hepatitis B for short)."
sentence += "Nitrofurans are broad-spectrum antibiotics that kill most gram-positive and Gram-negative bacteria, fungi, protozoa and other pathogens. They act on microbial enzyme system, inhibit acetyl-coA, interfere with microbial carbohydrate metabolism, and thus play A antibacterial role."
sentence += "Fibrinogen is one of the earliest coagulation factors found by human beings. It is an elongated ellipsoid. It is a dimer formed by three pairs of polypeptide chains (a pair of α chains, a pair of β chains, a pair of γ chains) connected by disulfide bonds, and its molecular weight is about 340,000."
## 用来存字符索引的
index_bufs = []
index_bufs.append(disease_classifier.get_sentence_index(sentence))
index_bufs.append(chemical_classifier.get_sentence_index(sentence))
index_bufs.append(genetic_classifier.get_sentence_index(sentence))

In [ ]:
## 按照以下方式来打印识别到的实体
for index_buf in index_bufs:
    for index_pair in index_buf:
        print("word:\""+ sentence[index_pair[0]:index_pair[1]] + "\" type:" + index_pair[2])

In [20]:
'''coding = utf-8'''
import os

# 分割为句子（简陋版本）
# 具体的思路就是读取内容，根据'.'来分割句子
# 对于一些小数点的数据就会出现问题
input_dir = os.path.abspath('.')
sentences = []
sentence = ""
for root, dirs, files in os.walk(os.path.join(input_dir,'dataset\\txt\\')):
    count = 1
    for file in files:
        line = ""
        fn = os.path.join(root,file)
        isAppend = False
        with open(fn,'r') as file_to_read:
            while True:
                line = file_to_read.readline()
                if not line:
                    ## 文件结尾退出文件
                    break
                else:
                    ## 空格行自动跳过
                    if( line == '\n'):
                        continue
                    start_index = 0
                    for index in range(len(line)):
                        if(line[index] == '.' 
                           or line[index] == ';' 
                           or line[index] == '?'
                           or line[index] == '!'):
                            if( start_index == index ):
                                start_index = index + 1
                                continue
                            if( isAppend ):
                                isAppend = False
                                sentence += line[start_index:index]
                            else:
                                sentence = line[start_index:index]
                            start_index = index + 1
                            sentences.append(sentence)
                        if(line[index] == '\n'):
                            if( not isAppend ):
                                isAppend = True
                            else:
                                sentence += ' '
                            sentence += line[start_index:index]    
        # 先处理一个文件试试
        break

In [21]:
for sentence in sentences:
    print(sentence)
## 感觉OK

Chapter 1 Hepatitis B Virus Infection: OverviewAbstract Hepatitis B virus (HBV) is a DNA virus, belonging to the Hepadnaviridae family
 It is a partially double-stranded DNA virus with a small viral genome Chronic HBV infection remains a global public health problem
 If leftuntreated, chronic HBV infection can progress to end-stage liver disease, such as liver cirrhosis and hepatocellular carcinoma (HCC)
 In recent years, tremendous advances in the field of HBV basic and clinical research have been achieved, ranging from the HBV biological characteristics, immunopathogenesis, and animal models to the development of new therapeutic strategies and new drugs againstHBV
 In this overview, we begin with a brief history of HBV discovery and treat-ment milestones
 We then briefly summarize the HBV research advances, whichwill be detailed in the following chapters
 1 Discovery of HBVAs early as the 1950s, clinicians observed the hepatitis that occurred after blood transfusion and proposed the 